In [1]:
#读取刚度矩阵
import numpy as np

# 读取数据文件，获取节点个数及自由度
def get_size(file_path):
    max_node = 0
    with open(file_path, 'r') as file:
        for line in file:
            data = line.split(',')
            node1 = int(data[0])
            node2 = int(data[2])
            max_node = max(max_node, node1, node2)
    return max_node * 6  # 这里我们假设每个节点都有6个自由度

# 从文件中读取稀疏矩阵数据
def read_data(file_path, size):
    matrix = np.zeros((size, size))
    with open(file_path, 'r') as file:
        for line in file:
            data = line.split(',')
            node1 = int(data[0])
            dof1 = int(data[1])
            node2 = int(data[2])
            dof2 = int(data[3])
            value = float(data[4])
            # 计算在刚度矩阵中的位置，注意我们这里索引要减1，因为Python是0基索引
            index1 = (node1 - 1) * 6 + dof1 - 1
            index2 = (node2 - 1) * 6 + dof2 - 1
            matrix[index1, index2] = value
    return matrix

file_path_K = "E:\phd\capytaine_test\FDM\DM-FEM\data\Job-1_largemesh_STIF1.mtx"  # 替换为你的文件路径
size_K = get_size(file_path_K)
matrix_K = read_data(file_path_K, size_K)

# 将读取到的下三角部分复制到上三角部分，恢复完整的刚度矩阵
for i in range(size_K):
    for j in range(i+1, size_K):
        matrix_K[i, j] = matrix_K[j, i]


file_path_M = "E:\phd\capytaine_test\FDM\DM-FEM\data\Job-1_largemesh_MASS1.mtx"  # 替换为你的文件路径
size_M = get_size(file_path_M)
matrix_M = read_data(file_path_M, size_M)

# 将读取到的下三角部分复制到上三角部分，恢复完整的刚度矩阵
for i in range(size_M):
    for j in range(i+1, size_M):
        matrix_M[i, j] = matrix_M[j, i]



In [3]:
import numpy as np

# Add a small positive number to the diagonal elements of the matrix
regularized_matrix_M = matrix_M + np.eye(matrix_M.shape[0]) * 1e-6

try:
    np.linalg.cholesky(regularized_matrix_M)
    print("Regularized matrix is positive definite")
except np.linalg.LinAlgError:
    print("Regularized matrix is still not positive definite")

Regularized matrix is positive definite


In [11]:
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigs
import numpy as np

# 将刚度矩阵和质量矩阵转换为稀疏矩阵
K = csr_matrix(matrix_K)
M = csr_matrix(regularized_matrix_M)

# 找到刚度矩阵和质量矩阵对角线元素的最大值
max_diag_K = max(K.diagonal())
max_diag_M = max(M.diagonal())

# 使用对角线元素的最大值对矩阵进行缩放
K = K / max_diag_K
M = M / max_diag_M

# 解决一般化特征值问题，只找到最小的20个特征值和对应的特征向量
try:
    eigenvalues, eigenvectors = eigs(K, 20, M, which='SM')
except Exception as e:
    print(f"Failed to solve the eigenvalue problem: {e}")
    eigenvalues = eigenvectors = None

if eigenvalues is not None:
    # 计算模态频率（需要对特征值开平方并取实部，因为在有阻尼的情况下，特征值可能是复数）
    frequencies = np.sqrt(np.real(eigenvalues))

    # 打印模态频率
    print(frequencies)


Failed to solve the eigenvalue problem: ARPACK error -1: No convergence (3781 iterations, 0/20 eigenvectors converged) [ARPACK error -14: DNAUPD  did not find any eigenvalues to sufficient accuracy.]


Regularized matrix is positive definite


In [6]:
import numpy as np
from scipy.sparse.linalg import eigsh
from scipy.sparse import csr_matrix

# 加载你的刚度矩阵和质量矩阵
# 假设你已经有了加载矩阵的函数get_stiffness_matrix()和get_mass_matrix()

# 将矩阵转换为CSR格式（稀疏矩阵格式），这是eigsh函数需要的格式
K = csr_matrix(matrix_K)
M = csr_matrix(regularized_matrix_M)

# 解决一般化特征值问题，只找到最小的50个特征值和对应的特征向量
eigenvalues, eigenvectors = eigsh(K, 10, M)

# 计算模态频率（需要对特征值开平方并取实部，因为在有阻尼的情况下，特征值可能是复数）
frequencies = np.sqrt(np.real(eigenvalues))

# 打印模态频率
print(frequencies)

# eigenvectors 就是你的模态振型，你可以对它进行进一步的分析或者可视化


[1.64324363e+08 1.64324363e+08 1.64324363e+08 1.64324363e+08
 1.64324363e+08 1.64324363e+08 1.64324363e+08 1.64324363e+08
 1.64324363e+08 1.64324363e+08]


In [ ]:
import numpy as np

# Add a small positive number to the diagonal elements of the matrix
regularized_matrix_M = matrix_M + np.eye(matrix_M.shape[0]) * 1e-6

try:
    np.linalg.cholesky(regularized_matrix_M)
    print("Regularized matrix is positive definite")
except np.linalg.LinAlgError:
    print("Regularized matrix is still not positive definite")

In [ ]:
# from scipy.linalg import eigh
# import numpy as np

# # 假设你已知质量矩阵M和刚度矩阵K
# # M = ...
# # K = ...

# # 求解广义特征值问题
# eigvals, eigvecs = eigh(matrix_K, regularized_matrix_M, eigvals=(0,19))

In [ ]:
import numpy as np
from scipy.sparse.linalg import eigs

# 这里，你的结构刚度矩阵是K，质量矩阵是M。
# K = ...
# M = ...

# 现在，我们使用eigsh函数来解决特征值问题。
# 这个函数默认使用'Lanczos'方法。
# 我们请求前20个最大的特征值（和相应的特征向量）。

num_eigvals = 20  # 所需的特征值数量

eigvals, eigvecs = eigs(matrix_K, k=num_eigvals, M=regularized_matrix_M)

# 因为我们处理的是振动问题，所以特征值应该是振动频率的平方。
# 所以，我们需要从特征值中得到振动频率。
# 请注意，我们假设这里的单位是正确的（即，K是在赫兹的平方中，M是在千克中）。

frequencies = np.sqrt(eigvals) / (2*np.pi)

# frequencies现在是前20个振动频率。


In [ ]:
eigvecs

In [ ]:
from scipy.sparse.linalg import eigsh
eigvals_large, eigvecs = eigsh(matrix_K, 20, matrix_M, which='LM')
eigvals = 1.0 / eigvals_large

In [ ]:
is_diagonal_positive = np.all(np.diag(matrix) > 0)
print("Are all diagonal elements positive?", is_diagonal_positive)

In [ ]:
np.savetxt("stiffness_matrix.csv", matrix, delimiter=",")

In [ ]:
import numpy as np

# 节点ID表，这些是你想要从大矩阵中取出的节点
node_ids = [381, 382, 443, 442]

# 索引，将节点ID转换为矩阵索引，因为每个节点有6个自由度
indices = []
for node_id in node_ids:
    for i in range(6):
        indices.append((node_id-1)*6 + i)

# 使用numpy的take函数从大矩阵中取出小矩阵
K_small = np.take(np.take(matrix, indices, axis=0), indices, axis=1)

# 现在K_small就是你需要的24x24的矩阵


In [ ]:
import numpy as np

# 节点ID表，这些是你想要从大矩阵中取出的节点
node_ids = [105, 106, 167, 166]

# 索引，将节点ID转换为矩阵索引，因为每个节点有6个自由度
indices = []
for node_id in node_ids:
    for i in range(6):
        indices.append((node_id-1)*6 + i)

# 使用numpy的take函数从大矩阵中取出小矩阵
K_small2 = np.take(np.take(matrix, indices, axis=0), indices, axis=1)

# 现在K_small就是你需要的24x24的矩阵
K_small==K_small2

In [ ]:
#验证导出的刚度矩阵是否正确，进行静态的计算
def apply_boundary_conditions(stiffness_matrix, boundary_nodes):
    """
    在刚度矩阵上应用边界条件。

    参数：
    stiffness_matrix (numpy.ndarray)：总刚度矩阵。
    boundary_nodes (list)：边界节点列表。

    返回：
    numpy.ndarray：应用了边界条件后的刚度矩阵。
    """
    # 复制原始刚度矩阵，以避免修改原始数据
    stiffness_matrix_bc = np.array(stiffness_matrix, copy=True)

    # 因为每个节点有6个自由度，所以需要对每个节点的6个自由度都进行处理
    for node_id in boundary_nodes:
        for i in range(6):
            dof_index = (node_id - 1) * 6 + i  # 计算自由度在矩阵中的位置
            stiffness_matrix_bc[:, dof_index] = 0  # 置零该自由度对应的列
            stiffness_matrix_bc[dof_index, :] = 0  # 置零该自由度对应的行
            stiffness_matrix_bc[dof_index, dof_index] = 1  # 在对角线位置上置1

    return stiffness_matrix_bc

def apply_load(load_nodes, load_direction, load_value, num_nodes):
    """
    创建载荷向量并应用载荷。

    参数：
    load_nodes (list)：载荷节点列表。
    load_direction (int)：载荷方向（1、2或3）。
    load_value (float)：载荷值。
    num_nodes (int)：节点总数。

    返回：
    numpy.ndarray：载荷向量。
    """
    load_vector = np.zeros(num_nodes * 6)  # 创建载荷向量
    for node_id in load_nodes:
        dof_index = (node_id - 1) * 6 + load_direction - 1  # 计算载荷在载荷向量中的位置
        load_vector[dof_index] += load_value  # 应用载荷

    return load_vector


In [ ]:
load_nodes = list(range(122, 794, 61))  # 需要将载荷应用的节点列表
load_direction = 3  # 载荷方向
load_value = 1.  # 载荷值
num_nodes = 793  # 节点总数
KKKKK  = apply_boundary_conditions(matrix, load_nodes)
load_vector = apply_load(load_nodes, load_direction, load_value, num_nodes)
U = np.linalg.solve(KKKKK, load_vector)


In [ ]:
Uz = U[2::6]